# Solutions for discussion January
## Single effect blocks for pymc3 and logistic
n_gene_in_block = 20, sample_size = 200000

In [ ]:
import pandas as pd, numpy as np
import os
import pyreadr
from dsc.dsc_io import load_rds
cwd = os.path.expanduser("~/GIT/cnv-gene-mapping/data")
folder = "deletion_simu_30_shape0.777_scale0.843"
# name = "deletion.genes.block20.for_simu.sample.combined.genes.block1"

In [ ]:
# +1 in gene index for gene name
fisher = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.genes.block1.fisher.gz", header = 0, sep = "\t")
fisher["p_deleterious"] = [k if i >= j else 1 for i,j,k in zip(fisher["d_c"], fisher["nd_c"], fisher["p"])]

In [3]:
fisher.shape

(2290, 7)

In [4]:
beta = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.block30.shape0.777.scale0.843.beta", header = None, names = ["beta"])

In [5]:
beta.tail()

,beta
22351,0.0
22352,0.0
22353,0.0
22354,0.0
22355,0.0


In [6]:
# index1 = pd.read_csv(f"{cwd}/deletion.block1.index.csv", header = None, sep = "\t", names = ["id1", "id2"])
index1 = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.block1.index.csv", header = None, sep = "\t", names = ["id1", "id2"])

In [7]:
index2 = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.block1.forsimu.index.csv", header = None, sep = "\t", names = ["is1", "is2"])

In [8]:
index2.tail()

,is1,is2
523,21904,21905
524,21936,21941
525,21979,21981
526,22088,22097
527,22337,22355


In [9]:
index = pd.concat([index1, index2], axis = 1)

In [10]:
index.tail()

,id1,id2,is1,is2
523,2250,2251,21904,21905
524,2252,2257,21936,21941
525,2258,2260,21979,21981
526,2261,2270,22088,22097
527,2271,2289,22337,22355


In [11]:
index["number_effect"] = [np.count_nonzero(beta.loc[item[0]:item[1]]["beta"].tolist()) for idx, item in index2.iterrows()]

In [12]:
idx_beta = index[index["number_effect"] == 1][["is1", "is2", "id1", "id2"]]

In [13]:
beta_in_block = list()
fisher_in_block = list()
d_c = list()
d_nc = list()
nd_c = list()
nd_nc = list()
gene_in_block = list()
for idx, item in idx_beta.iterrows():
    beta_in_block.append(beta["beta"][item[0]:item[1]+1].tolist())
    fisher_in_block.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["p"].tolist())
    d_c.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["d_c"].tolist())
    d_nc.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["d_nc"].tolist())
    nd_c.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["nd_c"].tolist())
    nd_nc.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["nd_nc"].tolist())
    gene_in_block.append([f"gene_{i+1}" for i in range(item[2], item[3]+1)])

In [14]:
index = index[index["number_effect"] == 1][["id1", "id2"]]
index = index.set_index([[i for i in range(index.shape[0])]])

In [15]:
index.shape

(69, 2)

In [29]:
index.head()

,id1,id2
0,0,7
1,23,36
2,51,53
3,140,144
4,146,164


In [17]:
logit_pymc3 = pd.DataFrame()
for idx, item in index.iterrows():
    logit = pyreadr.read_r(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.logit.rds")
    pymc3 = pd.read_csv(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.pymc3.chain_1.uniform_new_bound.gz", usecols = [0], 
                        sep = "\t", header = 0)
    pymc3_1 = pd.read_csv(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.pymc3.chain_1.uniform_new_bound.seed_1.gz", usecols = [0], 
                          sep = "\t", header = 0, names = ["pymc3_s1"])
    pymc3_2 = pd.read_csv(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.pymc3.chain_3.uniform_new_bound.gz", usecols = [0], 
                         sep = "\t", header = 0, names = ["pymc3_c3"])
    pymc3_3 = pd.read_csv(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.pymc3.chain_3.uniform_new_bound.seed_1.gz", usecols = [0], 
                         sep = "\t", header = 0, names = ["pymc3_s1_c3"])
    pymc3["block"] = f"block{idx}"
    susie = load_rds(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.SuSiE.L_10.prior_0p005.susie.rds")["pip"]
    res = pd.concat([logit[None][["p1"]], pymc3[["inclusion_probability", "block"]], pymc3_1[["pymc3_s1"]], pymc3_2[["pymc3_c3"]], pymc3_3[["pymc3_s1_c3"]]], axis = 1)
    res["susie"] = susie
    res["beta"] = beta_in_block[idx]
    res["fisher"] = fisher_in_block[idx]
    res["gene"] = gene_in_block[idx]
    res["d_c"] = d_c[idx]
    res["d_nc"] = d_nc[idx]
    res["nd_c"] = nd_c[idx]
    res["nd_nc"] = nd_nc[idx]
    res = res.rename(columns = {"p1": "logit", "inclusion_probability": "pymc3"})
    logit_pymc3 = pd.concat([logit_pymc3, res])

In [18]:
logit_pymc3.head()

,logit,pymc3,block,pymc3_s1,pymc3_c3,pymc3_s1_c3,susie,beta,fisher,gene,d_c,d_nc,nd_c,nd_nc
0,0.125,0.0145,block0,0.0125,0.076000,0.084000,0.0,0.000000,0.790466,gene_1,6,11922,8,11920
1,0.125,0.0120,block0,0.0150,0.074167,0.079333,0.0,0.000000,0.790466,gene_2,6,11922,8,11920
2,0.125,0.0170,block0,0.0190,0.077833,0.070167,0.0,0.000000,0.790466,gene_3,6,11922,8,11920
3,0.125,0.0110,block0,0.0150,0.071667,0.074500,0.0,0.000000,0.790466,gene_4,6,11922,8,11920
4,0.125,0.0160,block0,0.0170,0.071167,0.075000,0.0,0.552649,0.790466,gene_5,6,11922,8,11920


In [19]:
logit_pymc3["beta"] = logit_pymc3.apply(lambda x: 0 if x["beta"] == -0 else x["beta"], axis = 1)
logit_pymc3["beta"] = [np.round(x, 6) for x in logit_pymc3["beta"]]
logit_pymc3["logit"] = [np.round(x, 6) for x in logit_pymc3["logit"]]
logit_pymc3["pymc3"] = [np.round(x, 6) for x in logit_pymc3["pymc3"]]
logit_pymc3["pymc3_s1"] = [np.round(x, 6) for x in logit_pymc3["pymc3_s1"]]
logit_pymc3["pymc3_c3"] = [np.round(x, 6) for x in logit_pymc3["pymc3_c3"]]
logit_pymc3["pymc3_s1_c3"] = [np.round(x, 6) for x in logit_pymc3["pymc3_s1_c3"]]
logit_pymc3["susie"] = [np.round(x, 6) for x in logit_pymc3["susie"]]

In [20]:
logit_pymc3 = logit_pymc3[["block", "gene", "beta", "fisher", "d_c", "d_nc", "nd_c", "nd_nc", "logit", "pymc3", "pymc3_s1", "pymc3_c3", "pymc3_s1_c3", "susie"]]

In [21]:
logit_pymc3.head()

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,pymc3,pymc3_s1,pymc3_c3,pymc3_s1_c3,susie
0,block0,gene_1,0.000000,0.790466,6,11922,8,11920,0.125,0.0145,0.0125,0.076000,0.084000,0.0
1,block0,gene_2,0.000000,0.790466,6,11922,8,11920,0.125,0.0120,0.0150,0.074167,0.079333,0.0
2,block0,gene_3,0.000000,0.790466,6,11922,8,11920,0.125,0.0170,0.0190,0.077833,0.070167,0.0
3,block0,gene_4,0.000000,0.790466,6,11922,8,11920,0.125,0.0110,0.0150,0.071667,0.074500,0.0
4,block0,gene_5,0.552649,0.790466,6,11922,8,11920,0.125,0.0160,0.0170,0.071167,0.075000,0.0


In [28]:
logit_pymc3[logit_pymc3["block"] == "block3"]

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,pymc3,pymc3_s1,pymc3_c3,pymc3_s1_c3,susie
0,block3,gene_141,0.000000,2.034862e-16,145,11783,37,11891,0.249999,0.2480,0.2815,0.835500,0.600500,0.249999
1,block3,gene_142,0.000000,2.034862e-16,145,11783,37,11891,0.249999,0.3015,0.4325,0.538000,0.645667,0.249999
2,block3,gene_143,1.536428,2.034862e-16,145,11783,37,11891,0.249999,0.2240,0.2725,0.539833,0.668667,0.249999
3,block3,gene_144,0.000000,2.034862e-16,145,11783,37,11891,0.249999,0.3050,0.0815,0.560167,0.546500,0.249999
4,block3,gene_145,0.000000,2.981444e-11,97,11831,25,11903,0.000003,0.0120,0.0115,0.147167,0.138667,0.000005


In [23]:
logit_pymc3.to_csv(f"{cwd}/{folder}/PIP_comparison.csv", sep = "\t", header = True, index = False)

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,pymc3,pymc3_s1,pymc3_c3,pymc3_s1_c3,susie
0,block56,gene_1751,0.000000,7.743573e-01,7,11921,5,11923,0.000000,0.0215,0.0195,0.007667,0.005500,0.000000
1,block56,gene_1752,0.000000,3.627286e-03,32,11896,12,11916,0.000000,0.0210,0.0210,0.007833,0.007333,0.000000
2,block56,gene_1753,0.000000,6.987577e-10,159,11769,67,11861,0.001056,0.0100,0.0115,0.335333,0.670333,0.001288
3,block56,gene_1754,0.000000,6.987577e-10,159,11769,67,11861,0.001056,0.0100,0.0055,0.670333,0.264333,0.001288
4,block56,gene_1755,0.000000,6.987577e-10,159,11769,67,11861,0.001056,0.0125,0.0110,0.336000,0.670000,0.001288
5,block56,gene_1756,0.860409,1.164344e-12,203,11725,84,11844,0.498415,0.9955,0.0145,0.337833,0.686167,0.498067
6,block56,gene_1757,0.000000,1.164344e-12,203,11725,84,11844,0.498415,0.0205,1.0000,0.666500,0.060167,0.498067
7,block56,gene_1758,0.000000,4.680839e-01,122,11806,110,11818,0.000000,0.0065,0.0155,0.312167,0.424500,0.000000
8,block56,gene_1759,0.000000,4.680839e-01,122,11806,110,11818,0.000000,0.0105,0.0050,0.519000,0.554500,0.000000
9,block56,gene_1760,0.000000,5.250216e-02,83,11845,59,11869,0.000000,0.0055,0.0065,0.200500,0.351000,0.000000


In [27]:
sum(logit_pymc3[logit_pymc3["block"] == "block56"]["pymc3_s1_c3"])

4.937497999999999

In [79]:
logit_pymc3.tail()

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,pymc3,pymc3_s1,pymc3_c3,pymc3_s1_c3,susie
14,block68,gene_2286,0.0,0.209819,15,11913,8,11920,0.008967,0.0190,0.0225,0.153000,0.152333,0.015341
15,block68,gene_2287,0.0,0.035100,12,11916,3,11925,0.059251,0.0785,0.0830,0.103833,0.106667,0.057696
16,block68,gene_2288,0.0,0.035100,12,11916,3,11925,0.059251,0.0725,0.0750,0.100000,0.107167,0.057696
17,block68,gene_2289,0.0,0.035100,12,11916,3,11925,0.059251,0.0770,0.0740,0.103333,0.104167,0.057696
18,block68,gene_2290,0.0,0.035100,12,11916,3,11925,0.059251,0.0715,0.0745,0.102833,0.097333,0.057696


In [57]:
logit_pymc3[logit_pymc3["fisher"] == min(logit_pymc3["fisher"])]

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,pymc3,pymc3_seed1,susie
0,block3,gene_141,0.000000,2.034862e-16,145,11783,37,11891,0.249999,0.2480,0.2815,0.249999
1,block3,gene_142,0.000000,2.034862e-16,145,11783,37,11891,0.249999,0.3015,0.4325,0.249999
2,block3,gene_143,1.536428,2.034862e-16,145,11783,37,11891,0.249999,0.2240,0.2725,0.249999
3,block3,gene_144,0.000000,2.034862e-16,145,11783,37,11891,0.249999,0.3050,0.0815,0.249999
